In [1]:
import pandas as pd
import mne
import numpy as np
import time
import os
import matplotlib.pyplot as plt
import pyarrow.feather as feather
from mne.export import export_raw

In [2]:
root_path="C:\\Users\\bruno\\Desktop\\Diplomski\\"
edf_path = root_path + " \\edf_signals.edf"
feather_path = root_path + " \\edf_signals.feather"
csv_path = root_path + "edf_signals.csv"

In [3]:
read_times = {}
write_times = {}
file_sizes = {}

In [4]:
start_time = time.time()
data = mne.io.read_raw_edf(edf_path, preload=True)

data_raw = data.get_data()
data_channels = data.ch_names
signal = pd.DataFrame(data_raw)
signal = signal.T
signal.columns = data_channels

read_times['EDF'] = time.time() - start_time

Extracting EDF parameters from C:\Users\bruno\Desktop\Diplomski\38011864_B\edf_signals.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 16987999  =      0.000 ... 33975.998 secs...


C:\Users\bruno\AppData\Local\Temp\ipykernel_22252\3335567956.py:2: RuntimeWarning: Channel names are not unique, found duplicates for: {'Flow Patient'}. Applying running numbers for duplicates.
  data = mne.io.read_raw_edf(edf_path, preload=True)


In [5]:
file_sizes['EDF'] = os.path.getsize(edf_path)

In [6]:
start_time = time.time()
sfreq = 500.0 

n_channels = len(signal.columns)
ch_names = list(signal.columns)
ch_types = ['eeg'] * n_channels
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)

# Create the RawArray object
data = signal.to_numpy().T
raw = mne.io.RawArray(data, info)

# Save as EDF
export_raw('data.edf',raw, fmt='edf', overwrite=True)  # Use the export_raw function for EDF
write_times['EDF'] = time.time() - start_time

Creating RawArray with float64 data, n_channels=21, n_times=16988000
    Range : 0 ... 16987999 =      0.000 ... 33975.998 secs
Ready.
Overwriting existing file.


In [15]:
start_time = time.time()
table_data = feather.read_table(feather_path)
feather_df = table_data.to_pandas()
read_times['Feather'] = time.time() - start_time

In [16]:
file_sizes['Feather'] = os.path.getsize(feather_path)

In [17]:
start_time = time.time()
feather.write_feather(feather_df, root_path + "edf_signals.feather")
write_times['Feather'] = time.time() - start_time

In [ ]:
start_time = time.time()
csv_df = pd.read_csv(csv_path)
read_times['CSV'] = time.time() - start_time

In [ ]:
file_sizes['CSV'] = os.path.getsize(csv_path)

In [12]:
start_time = time.time()
signal.to_csv('edf_signals.csv', index=False)
write_times['CSV'] = time.time() - start_time

In [ ]:
labels = list(read_times.keys())
fig, ax = plt.subplots(1, 3, figsize=(15, 5))

ax[0].bar(labels, list(read_times.values()))
ax[0].set_title('Vrijeme čitanja (sekunde)')
ax[0].set_ylabel('Vrijeme (s)')

ax[1].bar(labels, list(write_times.values()))
ax[1].set_title('Vrijeme pisanja (sekunde)')
ax[1].set_ylabel('Vrijeme (s)')

ax[2].bar(labels, list(file_sizes.values()))
ax[2].set_title('Veličina datoteke (bajt)')
ax[2].set_ylabel('Veličina (B)')

plt.show()